# LUAD CPTAC analysis

In [1]:
import pandas as pd
import numpy as np
from msresist.pre_processing import preprocessing, preprocessCPTAC
from msresist.sequence_analysis import pYmotifs
from msresist.clustering import MassSpecClustering

In [2]:
# pd.set_option('display.max_colwidth', 1000)
# pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_columns', 1000)

## 1 - Import and Preprocess Data

In [3]:
X = preprocessing(CPTAC=True, log2T=True)

d = X.select_dtypes(include=['float64']).T
i = X.select_dtypes(include=['object'])

1096/52428 peptides were not found in the proteome.
51332


/home/marcc/resistance-MS/msresist/sequence_analysis.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Gene'] = names
/home/marcc/resistance-MS/msresist/sequence_analysis.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Sequence'] = motifs


In [4]:
X

,Sequence,Protein,Gene,Position,C3N.01799,C3N.01799.N,C3L.01890,C3L.01890.N,C3N.00572,C3N.00572.N,C3N.02423,C3N.02423.N,C3N.02729,C3N.02729.N,C3L.00263,C3L.00263.N,C3N.01410,C3N.01410.N,C3N.00578,C3N.00578.N,C3N.02587,C3N.02587.N,C3L.00893,C3L.00893.N,C3N.01488,C3N.01488.N,C3N.01413,C3N.01413.N,C3N.01030,C3N.01030.N,C3N.02588,C3N.02588.N,C3N.00552,C3N.00552.N,C3L.01889,C3L.01889.N,C3N.00169,C3N.00169.N,C3L.00422,C3L.00422.N,C3L.00083,C3L.00083.N,C3N.00551,C3N.00551.N,C3N.01842,C3N.02089,C3N.02089.N,C3L.01682,C3L.01682.N,C3N.01016,C3N.01016.N,C3N.00580,C3N.00580.N,X11LU013,C3N.02000,C3N.02000.N,C3N.01489,C3N.01489.N,C3N.00737,C3N.00737.N,C3N.01405,C3N.01405.N,C3N.02587.1,C3N.01416,C3N.01416.N,C3N.02149,C3N.02149.N,C3N.02424,C3N.02424.N,C3L.00279,C3L.00279.N,C3N.02379,C3L.00095,C3L.00095.N,C3N.02572,C3N.02572.N,C3N.00559,C3N.00559.N,C3N.00545,C3N.00545.N,X11LU016,C3L.02345,C3L.02345.N,C3L.00093,C3L.00093.N,C3L.00510,C3L.00510.N,C3L.00913,C3L.00913.N,X11LU022,C3L.00412,C3L.00412.N,C3N.01415,C3N.01415.N,C3N.00549,C3N.00549.N,C3N.02433,C3N.02433.N,C3N.01021,C3N.01021.N,C3N.00203,C3N.00203.N,C3L.01330,C3L.01330.N,C3N.00574,C3N.00574.N,X11LU035,C3N.01071,C3N.01071.N,C3N.00550,C3N.00550.N,C3N.00167,C3N.00167.N,C3L.00001,C3L.00001.N,C3N.01074,C3L.01632,C3L.01632.N,C3N.02155,C3N.02155.N,C3N.00217,C3N.00217.N,C3N.02002,C3N.02002.N,C3N.02145,C3N.02145.N,C3N.02586,C3N.02586.N,C3L.02219,C3L.02219.N,C3N.00180,C3N.00180.N,C3L.01862,C3N.00704,C3N.00704.N,C3N.02158,C3N.02158.N,C3N.02421,C3N.02421.N,C3N.00556,C3N.00556.N,C3N.00294,C3N.01072,C3N.01072.N,C3L.00094,C3L.00094.N,C3L.00144,C3L.00144.N,C3N.00223,C3N.00223.N,C3N.02380,C3L.01924,C3L.01924.N,C3N.00433,C3N.00433.N,C3L.01683,C3L.01683.N,C3N.00293,C3N.00293.N,C3N.02380.N,C3L.00009,C3L.00009.N,C3N.00546,C3N.00546.N,C3N.01414,C3N.01414.N,C3N.02087,C3N.02087.N,C3L.00140,C3L.00140.N,C3N.02529,C3N.02529.N,C3L.00368,C3L.00368.N,C3N.02067,C3N.02067.N,C3N.02422,C3L.00080,C3L.00080.N,C3N.00547,C3N.00547.N,C3N.01023,C3N.01023.N,C3N.01024,C3N.01024.N,C3L.00973,C3L.00973.N,C3L.02348,C3L.02348.N,C3L.02508,C3L.02508.N,C3N.00560,C3N.00560.N,C3L.00604,C3L.00604.N,C3N.00199,C3N.00199.N,C3N.00579,C3N.00579.N,C3N.02582,C3N.02582.N,C3N.02379.1,C3L.02350,C3L.02350.N,C3N.00738,C3N.00738.N,C3N.00959,C3N.00959.N,C3N.02003,C3N.02003.N,C3N.02379.N,C3N.00175,C3N.00175.N,C3N.01823,C3N.01823.N,C3L.02549,C3L.02549.N,C3L.02365,C3L.02365.N
0,AAAAAsGGGVS,NP_002577.2,PBX2,S146-p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.333072,-1.157027,1.091254,-0.108078,-1.077948,-1.782146,1.253047,-0.931758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAAAAsQQGSA,NP_001308122.1,TBL1XR1,S119-p,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.040414,-1.014198,0.822105,-0.375466,1.296330,-0.256019,1.091794,-0.452485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.512168,-1.336851,0.452910,-1.263757,1.508932,-0.708340,-0.026060,-2.216061,0.342936,-0.142478,-0.438031,0.422947,-1.116016,0.589940,-0.756644,-0.000108,-1.592715,0.742549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
distance_method = "PAM250"
ncl = 4
GMMweight = 2

MSC = MassSpecClustering(i, ncl, GMMweight=GMMweight, distance_method=distance_method).fit(d, "NA")